In [1]:
import pandas as pd
import numpy as np
import ast
import random

from torch import nn
import torch

from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import T5ForConditionalGeneration, T5ForConditionalGeneration
from transformers import AdamW

from torch.utils.data import DataLoader

from datasets import Dataset

from ray import tune
from ray.tune.schedulers import ASHAScheduler

from utils import perturb_test_sent, evaluate as evaluate_results

device = torch.device("cuda")

2022-08-05 00:33:14.585385: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/lily/lyf6/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180487213/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
SOURCE_PATH = ""

# Load train/test datasets
df      = pd.read_csv(SOURCE_PATH + "data/train_words.csv")
df_val  = pd.read_csv(SOURCE_PATH + "data/test_words.csv")

# Load vocab
f = open("TagalogStemmerPython/output/with_info.txt", "r", encoding='latin1')
f = f.readlines()
vocab_tl = set(ast.literal_eval(item.strip('\n'))['word'] for item in f)
vocab_tl = set(df['Target']).union(vocab_tl) # Add in vocab from dataframe
vocab_tl = set(df_val['Target']).union(vocab_tl) # Add in vocab from test dataframe

# Split into train and test
test_idx = random.sample(range(df.shape[0]), round(df.shape[0]/5))
df_train = df.loc[list(set(range(df.shape[0])).difference(test_idx))].reset_index(drop=True)
df_test  = df.loc[test_idx].reset_index(drop=True)

dataset_train = Dataset.from_pandas(df_train)
dataset_test  = Dataset.from_pandas(df_test)
dataset_val   = Dataset.from_pandas(df_val)

In [3]:
def train(model, dataloader, perturb, mse_weight, tokenizer, optimizer):
    model.train()
    loss, steps = 0.0, 0.0
    
    for batch in dataloader:
        model.zero_grad()

        inputs = encode(batch, tokenizer)
        output = model(input_ids      = inputs['input_ids'].to(device),
                       attention_mask = inputs['attention_mask'].to(device),
                       labels         = inputs['labels'].to(device))
        
        if perturb:
            perturb_batch = {}
            perturb_batch['Input']  = list(map(lambda x: perturb_test_sent(x, vocab_tl), batch['Input'])) 
            perturb_batch['Target'] = ['']*len(perturb_batch['Input'])
            inputs2 = encode(perturb_batch, tokenizer)
            output2 = model(input_ids      = inputs2['input_ids'].to(device),
                            attention_mask = inputs2['attention_mask'].to(device))
        
        if perturb:
            # Compute squared diff loss
            min_idx     = min(output.logits.shape[1], output2.logits.shape[1])
            diff_tensor = output.logits[:,:min_idx,:]-output2.logits[:,:min_idx,:]
            mse_loss    = torch.sqrt(torch.mean(diff_tensor**2)/output.logits.shape[0])
            
            # Compute total loss
            total_loss = (mse_weight*mse_loss)+((1-mse_weight)*output.loss)
            
            total_loss.backward()
            optimizer.step()
            loss += float(total_loss)
            
        else:
            output.loss.backward()
            optimizer.step()
            loss += float(output.loss)
        steps += 1
    
    return loss/steps

def evaluate(model, dataloader, tokenizer):
    loss, steps = 0.0, 0.0
    with torch.no_grad():
        for batch in dataloader:
    
            inputs = encode(batch, tokenizer)
            output = model(input_ids      = inputs['input_ids'].to(device),
                           attention_mask = inputs['attention_mask'].to(device),
                           labels         = inputs['labels'].to(device))
            loss += float(output.loss)
            steps += 1
    return loss/steps

def clean_word(s):
    return s.replace('<pad>','').replace('</s>','')

# Generate top 5 words per candidate
def generate_k_candidates(model, dataloader, tokenizer, k=5):
    result = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = encode(batch, tokenizer)
            output = model.generate(input_ids      = inputs['input_ids'].to(device),
                                    attention_mask = inputs['attention_mask'].to(device),
                                    num_return_sequences = k,
                                    num_beams = k)
            output = tokenizer.batch_decode(output)
            output = list(map(clean_word, output))
            result.append(output)
    return result

def initialize(use_bert, lr, eps):
    # Initialize tokenizers, model, loss, optimizer
    if use_bert:
        tokenizer = AutoTokenizer.from_pretrained("jcblaise/roberta-tagalog-large")
        model = AutoModelForMaskedLM.from_pretrained("jcblaise/roberta-tagalog-large").to(device)
    else:
        model = T5ForConditionalGeneration.from_pretrained("google/byt5-small").to(device)
        tokenizer = AutoTokenizer.from_pretrained("google/byt5-small",
                                                  output_scores=True,
                                                  output_hidden_states=True)

    nll_loss = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(),
                      lr = lr,  # args.learning_rate - default is 5e-5
                      eps = eps # args.adam_epsilon  - default is 1e-8.
                    )
    return model, tokenizer, nll_loss, optimizer



In [4]:
def encode(examples, tokenizer):
    batch_size = len(examples['Input'])
    inputs = examples['Input']
    targets = examples['Target']
    
    tokenized_inputs = tokenizer(inputs+targets, 
                                 return_tensors='pt', 
                                 padding=True)
    model_inputs = {}
    model_inputs['input_ids']      = tokenized_inputs['input_ids'][:batch_size]
    model_inputs['attention_mask'] = tokenized_inputs['attention_mask'][:batch_size]
    model_inputs['labels']         = tokenized_inputs['input_ids'][batch_size:]

    return model_inputs

In [5]:
def train_loop(args):
    
    PERTURB_FLAG = False  if 'perturb'        not in args else args['perturb']
    MSE_WEIGHT   = 0.5    if 'mse_weight'     not in args else args['mse_weight']
    EARLY_STOP   = False  if 'early_stop'     not in args else args['early_stop']
    MODEL_NAME   = False  if 'model_name'     not in args else args['model_name']
    EPS          = 1e-8   if 'eps'            not in args else args['eps'] #.sample()
    LR           = 1e-5   if 'lr'             not in args else args['lr'] #.sample()
    USE_BERT     = True   if 'use_bert'       not in args else args['use_bert']
    EPOCHS       = np.inf if 'epochs'         not in args else args['epochs'] #.sample()
    BATCH_SIZE   = 8      if 'batch_size'     not in args else args['batch_size'] #.sample()
    REPORT       = True   if 'report'         not in args else args['report']

    model, tokenizer, nll_loss, optimizer = initialize(use_bert=USE_BERT, lr=LR, eps=EPS)
    
    train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
    test_loader  = DataLoader(dataset_test,  batch_size=BATCH_SIZE, shuffle=True) # Actually "test" in our defn
    val_loader   = DataLoader(dataset_val,   batch_size=BATCH_SIZE, shuffle=True) # Actually "val" in our defn

    best_val_loss = np.inf
    epochs = 0

    while epochs<EPOCHS: 
        train_loss = train(model, train_loader, PERTURB_FLAG, MSE_WEIGHT, tokenizer, optimizer)
        val_loss   = evaluate(model, test_loader, tokenizer)
        epochs += 1
        if REPORT:
            tune.report(TRAIN_LOSS=train_loss, VAL_LOSS=val_loss)
        else:
            print(f"Epoch {epochs}; Train: {train_loss}; Test: {val_loss}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if MODEL_NAME != False:
                model.save_pretrained(f"models/{MODEL_NAME}")
        else:
            if EARLY_STOP:
                break
        
    if not REPORT:
        output_lst = generate_k_candidates(model, val_loader, tokenizer, 5)
        result_dict = evaluate_results(output_lst, dataset_val['Target'])
        print(result_dict)
    print("Finished Training")

### Filipino-Roberta (Cruz et al., 2021)

In [6]:
args = {
    "perturb": False,
    "use_bert": True,
    "early_stopping": False,
    "lr": tune.loguniform(1e-6, 1e-4),
    "eps": tune.loguniform(1e-9, 1e-6),
    "epochs": tune.choice([10,30,50,70]),
    "batch_size": tune.choice([2, 4, 8, 16])
}

scheduler = ASHAScheduler(
        max_t=70,
        grace_period=1,
        reduction_factor=2)
result = tune.run(
        tune.with_parameters(train_loop),
        resources_per_trial={"gpu": 1},
        config=args,
        metric="VAL_LOSS",
        mode="min",
        num_samples=25,
        scheduler=scheduler
    )

2022-08-01 00:28:06,405	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
(pid=1270143) 2022-08-01 00:28:10.362566: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Trial name,status,loc,batch_size,epochs,eps,lr
train_loop_5714f_00000,RUNNING,172.28.236.13:1270143,16,70,1.27747e-07,3.61535e-06
train_loop_5714f_00001,PENDING,,2,70,1.05685e-09,1.03349e-05
train_loop_5714f_00002,PENDING,,16,70,2.26121e-09,2.82399e-06
train_loop_5714f_00003,PENDING,,4,50,3.92518e-07,7.46347e-05
train_loop_5714f_00004,PENDING,,2,50,1.40704e-07,5.29084e-05
train_loop_5714f_00005,PENDING,,2,70,6.2983e-09,1.51619e-05
train_loop_5714f_00006,PENDING,,4,10,1.286e-07,4.13642e-05
train_loop_5714f_00007,PENDING,,2,70,9.72934e-09,4.49079e-06
train_loop_5714f_00008,PENDING,,2,30,9.96554e-08,1.40437e-06
train_loop_5714f_00009,PENDING,,2,70,4.22887e-09,3.20661e-06


(pid=1270178) 2022-08-01 00:28:19.566925: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Trial name,status,loc,batch_size,epochs,eps,lr
train_loop_5714f_00000,RUNNING,172.28.236.13:1270143,16,70,1.27747e-07,3.61535e-06
train_loop_5714f_00001,RUNNING,172.28.236.13:1270178,2,70,1.05685e-09,1.03349e-05
train_loop_5714f_00002,PENDING,,16,70,2.26121e-09,2.82399e-06
train_loop_5714f_00003,PENDING,,4,50,3.92518e-07,7.46347e-05
train_loop_5714f_00004,PENDING,,2,50,1.40704e-07,5.29084e-05
train_loop_5714f_00005,PENDING,,2,70,6.2983e-09,1.51619e-05
train_loop_5714f_00006,PENDING,,4,10,1.286e-07,4.13642e-05
train_loop_5714f_00007,PENDING,,2,70,9.72934e-09,4.49079e-06
train_loop_5714f_00008,PENDING,,2,30,9.96554e-08,1.40437e-06
train_loop_5714f_00009,PENDING,,2,70,4.22887e-09,3.20661e-06


(pid=1270214) 2022-08-01 00:28:28.507266: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


...

Result for train_loop_5714f_00003:
  TRAIN_LOSS: 0.050346365463767265
  VAL_LOSS: 3.7589760571718216
  date: 2022-08-01_00-37-12
  done: false
  experiment_id: 70816d46d32241a2a4a0f8f4a0dc8097
  hostname: ziva
  iterations_since_restore: 49
  node_ip: 172.28.236.13
  pid: 1270293
  time_since_restore: 508.727370262146
  time_this_iter_s: 8.560077905654907
  time_total_s: 508.727370262146
  timestamp: 1659328632
  timesteps_since_restore: 0
  training_iteration: 49
  trial_id: 5714f_00003
  warmup_time: 0.004239797592163086
  


Trial name,status,loc,batch_size,epochs,eps,lr,iter,total time (s),TRAIN_LOSS,VAL_LOSS
train_loop_5714f_00003,RUNNING,172.28.236.13:1270293,4,50,3.92518e-07,7.46347e-05,49,508.727,0.0503464,3.75898
train_loop_5714f_00000,TERMINATED,172.28.236.13:1270143,16,70,1.27747e-07,3.61535e-06,1,14.978,18.1179,15.6804
train_loop_5714f_00001,TERMINATED,172.28.236.13:1270178,2,70,1.05685e-09,1.03349e-05,2,77.5303,2.83856,2.99955
train_loop_5714f_00002,TERMINATED,172.28.236.13:1270214,16,70,2.26121e-09,2.82399e-06,1,15.4614,18.4918,16.8184
train_loop_5714f_00004,TERMINATED,172.28.236.13:1270371,2,50,1.40704e-07,5.29084e-05,2,48.8138,1.74318,2.63187
train_loop_5714f_00005,TERMINATED,172.28.236.13:1270447,2,70,6.2983e-09,1.51619e-05,2,50.1161,2.48826,3.03249
train_loop_5714f_00006,TERMINATED,172.28.236.13:1270525,4,10,1.286e-07,4.13642e-05,10,131.13,0.0741428,2.7231
train_loop_5714f_00007,TERMINATED,172.28.236.13:1270602,2,70,9.72934e-09,4.49079e-06,1,32.7719,11.9034,6.75141
train_loop_5714f_00008,TERMINATED,172.28.236.13:1270685,2,30,9.96554e-08,1.40437e-06,1,38.3676,15.729,13.8267
train_loop_5714f_00009,TERMINATED,172.28.236.13:1270687,2,70,4.22887e-09,3.20661e-06,1,33.4108,13.4755,8.92761


Result for train_loop_5714f_00003:
  TRAIN_LOSS: 0.05626105403429332
  VAL_LOSS: 3.6339030116796494
  date: 2022-08-01_00-37-22
  done: false
  experiment_id: 70816d46d32241a2a4a0f8f4a0dc8097
  hostname: ziva
  iterations_since_restore: 50
  node_ip: 172.28.236.13
  pid: 1270293
  time_since_restore: 518.2156114578247
  time_this_iter_s: 9.488241195678711
  time_total_s: 518.2156114578247
  timestamp: 1659328642
  timesteps_since_restore: 0
  training_iteration: 50
  trial_id: 5714f_00003
  warmup_time: 0.004239797592163086
  
Result for train_loop_5714f_00003:
  TRAIN_LOSS: 0.05626105403429332
  VAL_LOSS: 3.6339030116796494
  date: 2022-08-01_00-37-22
  done: true
  experiment_id: 70816d46d32241a2a4a0f8f4a0dc8097
  experiment_tag: 3_batch_size=4,epochs=50,eps=0.0000,lr=0.0001
  hostname: ziva
  iterations_since_restore: 50
  node_ip: 172.28.236.13
  pid: 1270293
  time_since_restore: 518.2156114578247
  time_this_iter_s: 9.488241195678711
  time_total_s: 518.2156114578247
  timestamp:

Trial name,status,loc,batch_size,epochs,eps,lr,iter,total time (s),TRAIN_LOSS,VAL_LOSS
train_loop_5714f_00000,TERMINATED,172.28.236.13:1270143,16,70,1.27747e-07,3.61535e-06,1,14.978,18.1179,15.6804
train_loop_5714f_00001,TERMINATED,172.28.236.13:1270178,2,70,1.05685e-09,1.03349e-05,2,77.5303,2.83856,2.99955
train_loop_5714f_00002,TERMINATED,172.28.236.13:1270214,16,70,2.26121e-09,2.82399e-06,1,15.4614,18.4918,16.8184
train_loop_5714f_00003,TERMINATED,172.28.236.13:1270293,4,50,3.92518e-07,7.46347e-05,50,518.216,0.0562611,3.6339
train_loop_5714f_00004,TERMINATED,172.28.236.13:1270371,2,50,1.40704e-07,5.29084e-05,2,48.8138,1.74318,2.63187
train_loop_5714f_00005,TERMINATED,172.28.236.13:1270447,2,70,6.2983e-09,1.51619e-05,2,50.1161,2.48826,3.03249
train_loop_5714f_00006,TERMINATED,172.28.236.13:1270525,4,10,1.286e-07,4.13642e-05,10,131.13,0.0741428,2.7231
train_loop_5714f_00007,TERMINATED,172.28.236.13:1270602,2,70,9.72934e-09,4.49079e-06,1,32.7719,11.9034,6.75141
train_loop_5714f_00008,TERMINATED,172.28.236.13:1270685,2,30,9.96554e-08,1.40437e-06,1,38.3676,15.729,13.8267
train_loop_5714f_00009,TERMINATED,172.28.236.13:1270687,2,70,4.22887e-09,3.20661e-06,1,33.4108,13.4755,8.92761


2022-08-01 00:37:22,262	INFO tune.py:747 -- Total run time: 555.86 seconds (555.61 seconds for the tuning loop).


In [7]:
args = {
        'perturb': False,
        'model_name': "Roberta",
        'early_stopping': False,
        'use_bert': True,
        'eps': 4.9016e-09,	
        'lr': 7.79675e-05,
        'batch_size': 2,
        'epochs': 30,
        'report': False
        }
train_loop(args)

{'best_dl': 7.26, 'max_dl': 11.18, 'avg_dl': 8.783999999999999, 'acc_1': 0.0, 'acc_3': 0.0, 'acc_5': 0.0, 'target_in_candidate': 0.0}
Finished Training


In [9]:
args = {
    "perturb": True,
    "use_bert": True,
    "early_stopping": False,
    "lr": tune.loguniform(1e-6, 1e-4),
    "eps": tune.loguniform(1e-9, 1e-6),
    "epochs": tune.choice([10,30,50,70]),
    "batch_size": tune.choice([2, 4, 8, 16])
}

scheduler = ASHAScheduler(
        max_t=70,
        grace_period=1,
        reduction_factor=2)
result = tune.run(
        tune.with_parameters(train_loop),
        resources_per_trial={"gpu": 1},
        config=args,
        metric="VAL_LOSS",
        mode="min",
        num_samples=25,
        scheduler=scheduler
    )

(pid=1273157) 2022-08-01 00:53:42.955817: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Trial name,status,loc,batch_size,epochs,eps,lr
train_loop_e81b4_00000,RUNNING,172.28.236.13:1273157,4,10,7.21627e-08,6.24332e-06
train_loop_e81b4_00001,PENDING,,4,50,5.63389e-08,1.30104e-05
train_loop_e81b4_00002,PENDING,,8,50,3.87786e-09,3.84543e-06
train_loop_e81b4_00003,PENDING,,8,70,1.12229e-08,7.81169e-05
train_loop_e81b4_00004,PENDING,,16,70,9.38678e-07,2.23225e-06
train_loop_e81b4_00005,PENDING,,4,30,7.41852e-08,2.86982e-06
train_loop_e81b4_00006,PENDING,,4,30,3.90186e-07,3.30154e-05
train_loop_e81b4_00007,PENDING,,8,30,1.55689e-09,2.23192e-06
train_loop_e81b4_00008,PENDING,,16,30,1.61957e-07,3.36251e-06
train_loop_e81b4_00009,PENDING,,16,10,1.86864e-08,3.29492e-06


(pid=1273234) 2022-08-01 00:53:55.070111: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Trial name,status,loc,batch_size,epochs,eps,lr
train_loop_e81b4_00000,RUNNING,172.28.236.13:1273157,4,10,7.21627e-08,6.24332e-06
train_loop_e81b4_00001,RUNNING,172.28.236.13:1273234,4,50,5.63389e-08,1.30104e-05
train_loop_e81b4_00002,PENDING,,8,50,3.87786e-09,3.84543e-06
train_loop_e81b4_00003,PENDING,,8,70,1.12229e-08,7.81169e-05
train_loop_e81b4_00004,PENDING,,16,70,9.38678e-07,2.23225e-06
train_loop_e81b4_00005,PENDING,,4,30,7.41852e-08,2.86982e-06
train_loop_e81b4_00006,PENDING,,4,30,3.90186e-07,3.30154e-05
train_loop_e81b4_00007,PENDING,,8,30,1.55689e-09,2.23192e-06
train_loop_e81b4_00008,PENDING,,16,30,1.61957e-07,3.36251e-06
train_loop_e81b4_00009,PENDING,,16,10,1.86864e-08,3.29492e-06


(pid=1273271) 2022-08-01 00:54:03.659045: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


...

Result for train_loop_e81b4_00003:
  TRAIN_LOSS: 0.039721305031449564
  VAL_LOSS: 2.3542217016220093
  date: 2022-08-01_01-04-30
  done: false
  experiment_id: 756ec0b2d242445bb23d157a7d0ff6e2
  hostname: ziva
  iterations_since_restore: 69
  node_ip: 172.28.236.13
  pid: 1273349
  time_since_restore: 612.5577721595764
  time_this_iter_s: 7.477595806121826
  time_total_s: 612.5577721595764
  timestamp: 1659330270
  timesteps_since_restore: 0
  training_iteration: 69
  trial_id: e81b4_00003
  warmup_time: 0.012286186218261719
  


Trial name,status,loc,batch_size,epochs,eps,lr,iter,total time (s),TRAIN_LOSS,VAL_LOSS
train_loop_e81b4_00003,RUNNING,172.28.236.13:1273349,8,70,1.12229e-08,7.81169e-05,69,612.558,0.0397213,2.35422
train_loop_e81b4_00000,TERMINATED,172.28.236.13:1273157,4,10,7.21627e-08,6.24332e-06,2,80.005,2.95839,4.02432
train_loop_e81b4_00001,TERMINATED,172.28.236.13:1273234,4,50,5.63389e-08,1.30104e-05,4,109.176,1.08752,2.67782
train_loop_e81b4_00002,TERMINATED,172.28.236.13:1273271,8,50,3.87786e-09,3.84543e-06,1,20.2996,8.59503,13.9942
train_loop_e81b4_00004,TERMINATED,172.28.236.13:1273426,16,70,9.38678e-07,2.23225e-06,1,17.1464,9.77222,17.4803
train_loop_e81b4_00005,TERMINATED,172.28.236.13:1273503,4,30,7.41852e-08,2.86982e-06,1,28.1947,8.36585,13.0723
train_loop_e81b4_00006,TERMINATED,172.28.236.13:1273580,4,30,3.90186e-07,3.30154e-05,4,80.7517,0.690427,2.6139
train_loop_e81b4_00007,TERMINATED,172.28.236.13:1273657,8,30,1.55689e-09,2.23192e-06,1,21.3981,9.22552,15.305
train_loop_e81b4_00008,TERMINATED,172.28.236.13:1273744,16,30,1.61957e-07,3.36251e-06,1,21.6083,9.39222,16.0132
train_loop_e81b4_00009,TERMINATED,172.28.236.13:1273746,16,10,1.86864e-08,3.29492e-06,1,22.8636,9.39689,16.173


Result for train_loop_e81b4_00003:
  TRAIN_LOSS: 0.03942725933607547
  VAL_LOSS: 2.464548245072365
  date: 2022-08-01_01-04-38
  done: true
  experiment_id: 756ec0b2d242445bb23d157a7d0ff6e2
  hostname: ziva
  iterations_since_restore: 70
  node_ip: 172.28.236.13
  pid: 1273349
  time_since_restore: 619.8936975002289
  time_this_iter_s: 7.335925340652466
  time_total_s: 619.8936975002289
  timestamp: 1659330278
  timesteps_since_restore: 0
  training_iteration: 70
  trial_id: e81b4_00003
  warmup_time: 0.012286186218261719
  


Trial name,status,loc,batch_size,epochs,eps,lr,iter,total time (s),TRAIN_LOSS,VAL_LOSS
train_loop_e81b4_00000,TERMINATED,172.28.236.13:1273157,4,10,7.21627e-08,6.24332e-06,2,80.005,2.95839,4.02432
train_loop_e81b4_00001,TERMINATED,172.28.236.13:1273234,4,50,5.63389e-08,1.30104e-05,4,109.176,1.08752,2.67782
train_loop_e81b4_00002,TERMINATED,172.28.236.13:1273271,8,50,3.87786e-09,3.84543e-06,1,20.2996,8.59503,13.9942
train_loop_e81b4_00003,TERMINATED,172.28.236.13:1273349,8,70,1.12229e-08,7.81169e-05,70,619.894,0.0394273,2.46455
train_loop_e81b4_00004,TERMINATED,172.28.236.13:1273426,16,70,9.38678e-07,2.23225e-06,1,17.1464,9.77222,17.4803
train_loop_e81b4_00005,TERMINATED,172.28.236.13:1273503,4,30,7.41852e-08,2.86982e-06,1,28.1947,8.36585,13.0723
train_loop_e81b4_00006,TERMINATED,172.28.236.13:1273580,4,30,3.90186e-07,3.30154e-05,4,80.7517,0.690427,2.6139
train_loop_e81b4_00007,TERMINATED,172.28.236.13:1273657,8,30,1.55689e-09,2.23192e-06,1,21.3981,9.22552,15.305
train_loop_e81b4_00008,TERMINATED,172.28.236.13:1273744,16,30,1.61957e-07,3.36251e-06,1,21.6083,9.39222,16.0132
train_loop_e81b4_00009,TERMINATED,172.28.236.13:1273746,16,10,1.86864e-08,3.29492e-06,1,22.8636,9.39689,16.173


2022-08-01 01:04:38,507	INFO tune.py:747 -- Total run time: 660.30 seconds (660.10 seconds for the tuning loop).


In [10]:
args = {
        'perturb': False,
        'model_name': "Roberta_Pi",
        'early_stopping': False,
        'use_bert': True,
        'eps': 1.36206e-09,
        'lr': 6.82384e-05,
        'batch_size': 16,
        'epochs': 30,
        'report': False
        }
train_loop(args)

{'best_dl': 5.571428571428571, 'max_dl': 35.857142857142854, 'avg_dl': 9.705357142857142, 'acc_1': 0.0, 'acc_3': 0.0, 'acc_5': 0.0, 'target_in_candidate': 0.0}
Finished Training


### ByT5 (Raffel et al., 2021)
#### Fixed epochs

In [11]:
args = {
    "perturb": False,
    "use_bert": False,
    "early_stopping": False,
    "lr": tune.loguniform(1e-6, 1e-4),
    "eps": tune.loguniform(1e-9, 1e-6),
    "epochs": tune.choice([10,30,50,70]),
    "batch_size": tune.choice([2, 4, 8, 16])
}

scheduler = ASHAScheduler(
        max_t=70,
        grace_period=1,
        reduction_factor=2)
result = tune.run(
        tune.with_parameters(train_loop),
        resources_per_trial={"gpu": 1},
        config=args,
        metric="VAL_LOSS",
        mode="min",
        num_samples=25,
        scheduler=scheduler
    )

(pid=1276099) 2022-08-01 01:13:01.074570: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Trial name,status,loc,batch_size,epochs,eps,lr
train_loop_9a673_00000,RUNNING,172.28.236.13:1276099,2,30,4.18895e-07,2.59866e-06
train_loop_9a673_00001,PENDING,,16,50,8.32389e-08,4.24536e-06
train_loop_9a673_00002,PENDING,,4,30,2.31624e-09,1.50666e-05
train_loop_9a673_00003,PENDING,,8,30,3.76304e-09,5.96182e-05
train_loop_9a673_00004,PENDING,,2,70,1.23999e-07,8.86701e-05
train_loop_9a673_00005,PENDING,,16,10,1.78641e-07,5.56002e-06
train_loop_9a673_00006,PENDING,,4,50,1.12628e-08,1.73285e-06
train_loop_9a673_00007,PENDING,,8,70,7.17825e-07,8.70394e-06
train_loop_9a673_00008,PENDING,,2,70,2.03409e-09,4.95585e-05
train_loop_9a673_00009,PENDING,,4,10,1.47242e-08,5.2597e-05


(pid=1276135) 2022-08-01 01:13:14.648192: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Trial name,status,loc,batch_size,epochs,eps,lr
train_loop_9a673_00000,RUNNING,172.28.236.13:1276099,2,30,4.18895e-07,2.59866e-06
train_loop_9a673_00001,RUNNING,172.28.236.13:1276135,16,50,8.32389e-08,4.24536e-06
train_loop_9a673_00002,PENDING,,4,30,2.31624e-09,1.50666e-05
train_loop_9a673_00003,PENDING,,8,30,3.76304e-09,5.96182e-05
train_loop_9a673_00004,PENDING,,2,70,1.23999e-07,8.86701e-05
train_loop_9a673_00005,PENDING,,16,10,1.78641e-07,5.56002e-06
train_loop_9a673_00006,PENDING,,4,50,1.12628e-08,1.73285e-06
train_loop_9a673_00007,PENDING,,8,70,7.17825e-07,8.70394e-06
train_loop_9a673_00008,PENDING,,2,70,2.03409e-09,4.95585e-05
train_loop_9a673_00009,PENDING,,4,10,1.47242e-08,5.2597e-05


(pid=1276174) 2022-08-01 01:13:30.576986: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


...

Trial name,status,loc,batch_size,epochs,eps,lr,iter,total time (s),TRAIN_LOSS,VAL_LOSS
train_loop_9a673_00004,RUNNING,172.28.236.13:1276250,2,70,1.23999e-07,8.86701e-05,69,983.189,0.0404133,0.484663
train_loop_9a673_00000,TERMINATED,172.28.236.13:1276099,2,30,4.18895e-07,2.59866e-06,2,87.4654,16.3642,18.0947
train_loop_9a673_00001,TERMINATED,172.28.236.13:1276135,16,50,8.32389e-08,4.24536e-06,1,16.0707,36.4778,40.6175
train_loop_9a673_00002,TERMINATED,172.28.236.13:1276174,4,30,2.31624e-09,1.50666e-05,1,22.4825,24.4941,21.7868
train_loop_9a673_00003,TERMINATED,172.28.236.13:1276212,8,30,3.76304e-09,5.96182e-05,1,17.7485,29.0478,30.3274
train_loop_9a673_00005,TERMINATED,172.28.236.13:1276329,16,10,1.78641e-07,5.56002e-06,1,16.4728,36.0729,42.5028
train_loop_9a673_00006,TERMINATED,172.28.236.13:1276366,4,50,1.12628e-08,1.73285e-06,2,30.6718,24.8872,24.3425
train_loop_9a673_00007,TERMINATED,172.28.236.13:1276403,8,70,7.17825e-07,8.70394e-06,1,19.1427,31.6694,36.2082
train_loop_9a673_00008,TERMINATED,172.28.236.13:1276447,2,70,2.03409e-09,4.95585e-05,4,97.1714,3.79457,3.36176
train_loop_9a673_00009,TERMINATED,172.28.236.13:1276448,4,10,1.47242e-08,5.2597e-05,2,36.6567,19.4205,20.0378


Trial name,status,loc,batch_size,epochs,eps,lr,iter,total time (s),TRAIN_LOSS,VAL_LOSS
train_loop_9a673_00004,RUNNING,172.28.236.13:1276250,2,70,1.23999e-07,8.86701e-05,69,983.189,0.0404133,0.484663
train_loop_9a673_00000,TERMINATED,172.28.236.13:1276099,2,30,4.18895e-07,2.59866e-06,2,87.4654,16.3642,18.0947
train_loop_9a673_00001,TERMINATED,172.28.236.13:1276135,16,50,8.32389e-08,4.24536e-06,1,16.0707,36.4778,40.6175
train_loop_9a673_00002,TERMINATED,172.28.236.13:1276174,4,30,2.31624e-09,1.50666e-05,1,22.4825,24.4941,21.7868
train_loop_9a673_00003,TERMINATED,172.28.236.13:1276212,8,30,3.76304e-09,5.96182e-05,1,17.7485,29.0478,30.3274
train_loop_9a673_00005,TERMINATED,172.28.236.13:1276329,16,10,1.78641e-07,5.56002e-06,1,16.4728,36.0729,42.5028
train_loop_9a673_00006,TERMINATED,172.28.236.13:1276366,4,50,1.12628e-08,1.73285e-06,2,30.6718,24.8872,24.3425
train_loop_9a673_00007,TERMINATED,172.28.236.13:1276403,8,70,7.17825e-07,8.70394e-06,1,19.1427,31.6694,36.2082
train_loop_9a673_00008,TERMINATED,172.28.236.13:1276447,2,70,2.03409e-09,4.95585e-05,4,97.1714,3.79457,3.36176
train_loop_9a673_00009,TERMINATED,172.28.236.13:1276448,4,10,1.47242e-08,5.2597e-05,2,36.6567,19.4205,20.0378


Result for train_loop_9a673_00004:
  TRAIN_LOSS: 0.04002303977796138
  VAL_LOSS: 0.5288051571246357
  date: 2022-08-01_01-30-30
  done: true
  experiment_id: 5a4e0d0cbe974b6da42ff72d07876fce
  hostname: ziva
  iterations_since_restore: 70
  node_ip: 172.28.236.13
  pid: 1276250
  time_since_restore: 996.7339570522308
  time_this_iter_s: 13.545138835906982
  time_total_s: 996.7339570522308
  timestamp: 1659331830
  timesteps_since_restore: 0
  training_iteration: 70
  trial_id: 9a673_00004
  warmup_time: 0.004901409149169922
  


Trial name,status,loc,batch_size,epochs,eps,lr,iter,total time (s),TRAIN_LOSS,VAL_LOSS
train_loop_9a673_00000,TERMINATED,172.28.236.13:1276099,2,30,4.18895e-07,2.59866e-06,2,87.4654,16.3642,18.0947
train_loop_9a673_00001,TERMINATED,172.28.236.13:1276135,16,50,8.32389e-08,4.24536e-06,1,16.0707,36.4778,40.6175
train_loop_9a673_00002,TERMINATED,172.28.236.13:1276174,4,30,2.31624e-09,1.50666e-05,1,22.4825,24.4941,21.7868
train_loop_9a673_00003,TERMINATED,172.28.236.13:1276212,8,30,3.76304e-09,5.96182e-05,1,17.7485,29.0478,30.3274
train_loop_9a673_00004,TERMINATED,172.28.236.13:1276250,2,70,1.23999e-07,8.86701e-05,70,996.734,0.040023,0.528805
train_loop_9a673_00005,TERMINATED,172.28.236.13:1276329,16,10,1.78641e-07,5.56002e-06,1,16.4728,36.0729,42.5028
train_loop_9a673_00006,TERMINATED,172.28.236.13:1276366,4,50,1.12628e-08,1.73285e-06,2,30.6718,24.8872,24.3425
train_loop_9a673_00007,TERMINATED,172.28.236.13:1276403,8,70,7.17825e-07,8.70394e-06,1,19.1427,31.6694,36.2082
train_loop_9a673_00008,TERMINATED,172.28.236.13:1276447,2,70,2.03409e-09,4.95585e-05,4,97.1714,3.79457,3.36176
train_loop_9a673_00009,TERMINATED,172.28.236.13:1276448,4,10,1.47242e-08,5.2597e-05,2,36.6567,19.4205,20.0378


2022-08-01 01:30:30,801	INFO tune.py:747 -- Total run time: 1054.47 seconds (1054.30 seconds for the tuning loop).


In [6]:
args = {
        'perturb': False,
        'model_name': "ByT5",
        'early_stopping': False,
        'use_bert': True,
        'eps': 1.23999e-07,
        'lr': 8.86701e-05,
        'batch_size': 2,
        'epochs': 70,
        'report': False
        }
train_loop(args)

RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW

# Old Results

In [ ]:
best_val_loss = np.inf
epochs = 0

while epochs<70: 
    train_loss = train()
    val_loss   = evaluate()
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs += 1
        print(f"Epoch {epochs}; Train: {train_loss}; Test: {val_loss}")
    else:
        print(train_loss, val_loss, epochs)
        # break

In [18]:
# Epoch 46
# Results for normal training setting
output_lst = generate_k_candidates(df_val, 5)
evaluate_results(output_lst, df_val[1])

{'best_dl': 0.38,
 'max_dl': 3.14,
 'avg_dl': 1.839999999999999,
 'acc_1': 0.52,
 'acc_3': 0.65,
 'acc_5': 0.73,
 'target_in_candidate': 0.73}

Normal Training Setting

In [9]:
model, tokenizer, nll_loss, optimizer = initialize(USE_BERT=False)

best_val_loss = np.inf
epochs = 0

while True:
    train_loss = train()
    val_loss   = evaluate()
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs += 1
        print(f"Epoch {epochs}; Train: {train_loss}; Test: {val_loss}")
    else:
        print(train_loss, val_loss, epochs)
        break

Epoch 1; Train: 6.208001191951027; Test: 3.793813467025757
Epoch 2; Train: 2.9353004282663675; Test: 2.5620150566101074
Epoch 3; Train: 2.3688321778596926; Test: 2.276390314102173
Epoch 4; Train: 1.9996028182920345; Test: 1.9397112131118774
Epoch 5; Train: 1.7801968050150832; Test: 1.8884611129760742
Epoch 6; Train: 1.5986439016732303; Test: 1.5805171728134155
Epoch 7; Train: 1.3676149255234349; Test: 1.4615607261657715
Epoch 8; Train: 1.1776631346418838; Test: 1.3617585897445679
Epoch 9; Train: 1.0113097660643748; Test: 1.2640570402145386
Epoch 10; Train: 0.8564147209394569; Test: 1.1551393270492554
Epoch 11; Train: 0.7850987756363124; Test: 1.0066871643066406
Epoch 12; Train: 0.7518448811024427; Test: 0.9231386184692383
Epoch 13; Train: 0.6079841941087083; Test: 0.8400849103927612
0.5187350302734528 tensor(0.8553, device='cuda:0') 13


In [23]:
# Results for normal training setting
output_lst = generate_k_candidates(df_val, 5)
evaluate_results(output_lst, df_val[1])

{'best_dl': 1.19,
 'max_dl': 4.44,
 'avg_dl': 2.8239999999999994,
 'acc_1': 0.27,
 'acc_3': 0.45,
 'acc_5': 0.52,
 'target_in_candidate': 0.52}

Semi-Supervised Setting

In [59]:
model, tokenizer, nll_loss, optimizer = initialize(USE_BERT=False)

best_val_loss = np.inf
epochs = 0

while True:
    train_loss = train(True, 0.4)
    val_loss   = evaluate()
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs += 1
        print(f"Epoch {epochs}; Train: {train_loss}; Test: {val_loss}")
    else:
        print(train_loss, val_loss, epochs)
        break

Epoch 1; Train: 11.056835768636594; Test: 4.690049648284912
Epoch 2; Train: 7.1538529445317165; Test: 3.3120205402374268
Epoch 3; Train: 4.94390930420111; Test: 2.469358444213867
Epoch 4; Train: 3.9842169107484424; Test: 2.3854293823242188
Epoch 5; Train: 2.888489823696042; Test: 2.349212646484375
Epoch 6; Train: 2.4572199257937344; Test: 2.2144558429718018
Epoch 7; Train: 2.136661278314827; Test: 2.103837490081787
Epoch 8; Train: 1.9581665795696668; Test: 2.041865825653076
Epoch 9; Train: 1.793297808771291; Test: 1.8708345890045166
Epoch 10; Train: 1.7174554580499317; Test: 1.824424386024475
Epoch 11; Train: 1.5974195185771658; Test: 1.7025080919265747
Epoch 12; Train: 1.508887403021174; Test: 1.5896350145339966
Epoch 13; Train: 1.4078386083614727; Test: 1.4664043188095093
Epoch 14; Train: 1.3340068538326861; Test: 1.3377220630645752
Epoch 15; Train: 1.2421415347205707; Test: 1.225014090538025
Epoch 16; Train: 1.1677047228517612; Test: 1.1890171766281128
Epoch 17; Train: 1.10827949002

In [60]:
# Results for semi-supervised training setting
output_lst = generate_k_candidates(df_val, 5)
evaluate_results(output_lst, df_val[1])

{'best_dl': 0.94,
 'max_dl': 4.27,
 'avg_dl': 2.7039999999999993,
 'acc_1': 0.31,
 'acc_3': 0.46,
 'acc_5': 0.54,
 'target_in_candidate': 0.54}